In [ ]:
import os
from functools import partial
import pandas as pd
import numpy as np

from madrigal.evaluate.predict import get_drugbank_scores_wrapper
from madrigal.utils import BASE_DIR

# Utils

In [ ]:
drug_metadata = pd.read_pickle(os.path.join(BASE_DIR, 'processed_data/views_features_new/combined_metadata_ddi.pkl'))
drug_metadata['view_str'] = 1
print(drug_metadata.shape[0])

drugbank_ddi_classes = pd.read_pickle(BASE_DIR + "processed_data/polypharmacy_new/DrugBank/drugbank_ddi_directed_final_label_map.pkl")
drugbank_ddi_df = pd.read_csv(BASE_DIR + "processed_data/polypharmacy_new/DrugBank/drugbank_ddi_directed_final.tsv", index_col=0)
get_drugbank_scores = partial(get_drugbank_scores_wrapper, ckpt_list=['drawn-grass-4', 'misty-oath-5', 'whole-fog-7', 'snowy-serenity-8', 'revived-aardvark-8'], drugbank_ddi_classes=drugbank_ddi_classes)

21842


# Make predictions

In [3]:
combos = [
    ["Doxycycline", "Digoxin"],
    ["Doxycycline", "Warfarin"],
    ["Doxycycline", "Tacrolimus"],
    ["Doxycycline", "Levetiracetam"],
    ["Doxycycline", "Piracetam"],
]
outcomes = [
    'excretion rate, decrease | serum level, increase',
    'excretion, decrease',
    'serum level of the active metabolites, increase',
    'serum level, increase',
]

In [4]:
drug_names = np.unique(combos)
drug_inds = [drug_metadata[drug_metadata["node_name"] == drug_name].index.values[0] for drug_name in drug_names]

drug_1_ind_inds = [drug_names.tolist().index(pair[0]) for pair in combos]
drug_2_ind_inds = [drug_names.tolist().index(pair[1]) for pair in combos]

In [21]:
import warnings
import contextlib
import io
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    with contextlib.redirect_stdout(io.StringIO()):
        drugbank_scores = get_drugbank_scores(
            outcome_drugbank_inds=[drugbank_ddi_classes.tolist().index(o) for o in outcomes], 
            drug_inds=drug_inds, 
        )

In [22]:
pd.DataFrame(drugbank_scores[:, drug_1_ind_inds, drug_2_ind_inds], index=outcomes, columns=list(zip(np.array(drug_names)[drug_1_ind_inds], np.array(drug_names)[drug_2_ind_inds]))).drop_duplicates()

,"(Doxycycline, Digoxin)","(Doxycycline, Warfarin)","(Doxycycline, Tacrolimus)","(Doxycycline, Levetiracetam)","(Doxycycline, Piracetam)"
"excretion rate, decrease | serum level, increase",0.963690,0.949302,0.945904,0.013773,0.988898
"excretion, decrease",0.742310,0.800299,0.777249,0.219229,0.642533
"serum level of the active metabolites, increase",0.064820,0.309553,0.111543,0.119407,0.269221
"serum level, increase",0.375867,0.477788,0.615342,0.024048,0.106242


In [ ]:
# NOTE: The following file is 80GB large (see https://github.com/mims-harvard/Madrigal?tab=readme-ov-file#-notebooks for details)
normalized_rank_drugbank = np.load(BASE_DIR + "model_output/DrugBank/split_by_pairs/full_full_all_ddi_DrugBank_nash_drugs_normalized_ranks_700_5_runs_gmean_renormalized.npy", mmap_mode="r")
pd.DataFrame(normalized_rank_drugbank[[drugbank_ddi_classes.tolist().index(o) for o in outcomes], :, :][:, np.array(drug_inds)[drug_1_ind_inds], np.array(drug_inds)[drug_2_ind_inds]], index=outcomes, columns=list(zip(np.array(drug_names)[drug_1_ind_inds], np.array(drug_names)[drug_2_ind_inds]))).drop_duplicates()

,"(Doxycycline, Digoxin)","(Doxycycline, Warfarin)","(Doxycycline, Tacrolimus)","(Doxycycline, Levetiracetam)","(Doxycycline, Piracetam)"
"excretion rate, decrease | serum level, increase",0.997997,0.996835,0.996800,0.394416,0.999728
"excretion, decrease",0.997046,0.998859,0.998453,0.821208,0.993314
"serum level of the active metabolites, increase",0.561261,0.912119,0.790037,0.709302,0.904651
"serum level, increase",0.942572,0.960703,0.978966,0.420068,0.765712
